In [1]:
train.data <- read.csv(file.path("..", "data", "training_data.csv"))
test.data <- read.csv(file.path("..", "data", "test_data.csv"))

In [2]:
set.seed(100)
numeric.intensity <- as.numeric(train.data$Intensity)

x <- train.data[, -c(2,3)]
x$Intensity <- numeric.intensity
idx.zero.var <- apply(x, 2, var) == 0


x <- x[,!idx.zero.var]
y <- train.data$VALENCE.PLEASANTNESS

x.train = x
y.train = y

# x = predicteurs avec Intensity en numeric 
# y = que la réponse

#Test data : tout sauf les 
test <- test.data[,-c(1,2)]
test$Intensity <- as.numeric(test.data$Intensity)
test <- test[,!idx.zero.var]

In [28]:
library(xgboost)
dtrain = xgb.DMatrix(data =  as.matrix(x.train), label = y.train)


In [4]:
#CROSS-VALIDATION
best_param <- list()
best_seednumber <- 1234
best_rmse <- Inf
best_rmse_index <- 0

set.seed(123)
for (iter in 1:50) {
  param <- list(objective = "reg:squarederror",
                eval_metric = "rmse",
                max_depth = sample(3:7, 1),
                eta = runif(1, .01, .3))
  cv.nround <-  1000
  cv.nfold <-  5 # 5-fold cross-validation
  seed.number  <-  sample.int(10000, 1) # set seed for the cv
  set.seed(seed.number)
  mdcv <- xgb.cv(data = dtrain, params = param,  
                 nfold = cv.nfold, nrounds = cv.nround,
                 verbose = F, early_stopping_rounds = 8, maximize = FALSE)
  min_rmse_index  <-  mdcv$best_iteration
  min_rmse <-  mdcv$evaluation_log[min_rmse_index]$test_rmse_mean

      if (min_rmse < best_rmse) {
        best_rmse <- min_rmse
        best_rmse_index <- min_rmse_index
        best_seednumber <- seed.number
        best_param <- param
      }
}
nround = best_rmse_index
set.seed(best_seednumber)

ERROR: Error in eval(expr, envir, enclos): objet 'best_param' introuvable


In [8]:
xg_mod <- xgboost(data = dtrain, params = best_param, nround = nround, verbose = F)

In [33]:
test = test[,c(which(colnames(test)=="Intensity"),which(colnames(test)!="Intensity"))]
colnames(test) <- NULL

In [34]:
predictions = predict(xg_mod, xgb.DMatrix(data = as.matrix(test)))

In [36]:
submission <- data.frame(Id = 1:68, VALENCE.PLEASANTNESS = predictions)
write.csv(submission, file = "../Submissions/boosting_cv.csv", row.names = FALSE)